# If you want to save the Trasnformers checkpoints to your drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Dependencies

In [ ]:
! pip install "transformers==3.3.0"
! pip install gcsfs
! pip install pytrec_eval

     |████████████████████████████████| 1.1MB 6.2MB/s 
     |████████████████████████████████| 3.0MB 14.7MB/s 
     |████████████████████████████████| 1.2MB 35.1MB/s 
     |████████████████████████████████| 901kB 32.3MB/s 
     |████████████████████████████████| 112kB 7.5MB/s 
     |████████████████████████████████| 1.3MB 10.5MB/s 
     |████████████████████████████████| 143kB 27.8MB/s 
     |████████████████████████████████| 296kB 18.9MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=266837 sha256=9ea402c4358a5c068945457271a87911a15fde6476ebe442e60b3eac3f8b6536
  Stored in directory: /root/.cache/pip/wheels/55/66/40/1779aa0a8eb66e088669befe286f695cdfe420ba91ce662127
Successfully built pytrec-eval


**Get the code**

In [ ]:
import sys
sys.path.pop()

In [ ]:
import sys
%cd /content/
!test -d ExactMatchMarking || git clone https://github.com/BOUALILILila/ExactMatchMarking.git
if not 'ExactMatchMarking' in sys.path:
  sys.path += ['ExactMatchMarking']

/content
rm: cannot remove 'ExactMatchMarking': No such file or directory
Cloning into 'ExactMatchMarking'...
remote: Enumerating objects: 510, done.
remote: Counting objects: 100% (510/510), done.
remote: Compressing objects: 100% (340/340), done.
remote: Total 510 (delta 359), reused 318 (delta 167), pack-reused 0
Receiving objects: 100% (510/510), 123.02 KiB | 2.37 MiB/s, done.
Resolving deltas: 100% (359/359), done.


**Google Cloud Storage set up**

In [ ]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [ ]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
import os
os.environ['COLAB_SKIP_TPU_AUTH'] = '1'
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_ID="YOUR GCS PROJECT ID #@param {type:"string"}
bucket_name="BUCKET NAME" #@param {type:"string"}
!gcloud config set project {project_ID} 

Updated property [core/project].


# Zero-shot trasnfer setting

## Parameters

In [ ]:
## Tain params
MODEL_NAME_OR_PATH = "google/electra-base-discriminator" #@param {type:"string"}
# MODEL_TYPE = "electra" #@param {type:"string"}
CHECKPOINT_DIR='/path/to/ckpts' #@param {type:"string"}

OUTPUT_DIR = "/path/to/save/predictions" #@param {type:"string"}
assert OUTPUT_DIR, 'Must specify an existing GCS bucket name'
tf.io.gfile.makedirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Now we need to specify the input data dir. Should contain the .tfrecord files 
# and the supporting query-docids mapping files.


COLLECTION='robust04 ' #@param {type:"string"}

STRATEGY= "sim_pair" #@param {type:"string"}

## Train
TRAIN_DATA_DIR = "" #@param {type:"string"}
#  Convention:  {TRAIN_DATA_DIR}/ dataset_train_{TRAIN_SET_NAME}.tf
TRAIN_SET_NAME= "" #@param {type:"string"}

## Eval
EVAL_DATA_DIR = "gs://path/to/folder/containing/test/datasets/tfrecord/GCS" #@param {type:"string"}

# Convention: {EVAL_DATA_DIR}/dataset_dev_{EVAL_SET_NAME}.tf
EVAL_SET_NAME= "" #@param {type:"string"}
EVAL_QRELS_FILE_NAME = ""#@param {type:"string"}

# Convention: {EVAL_DATA_DIR}/dataset_test_{TEST_SET_NAME}.tf
TEST_SET_NAME = "robust04_description_doc_body_1000_p150_o75_sim_pair_doc_level_marking"#@param {type:"string"}
TEST_QRELS_FILE_NAME = ""#@param {type:"string"}
# output file name :
#   predict: predictions_{TEST_SET_NAME}_{OUT_SUFFIX}-{step}
#   eval: predictions_{OUT_SUFFIX}_{step}
OUT_SUFFIX = "msmarco-ft"#@param {type:"string"}

# Must be on GCS
LOG_DIR = "gs://path/to/log/dir/on/GCS"#@param {type:"string"}

***** Model output directory: gs://lila_data/output_dir/zero-shot/robust04/electra *****


## Run the model

In [ ]:
# coding=utf-8
from absl import logging as logger
import os, glob, re
import pandas as pd
import tensorflow as tf
from transformers import (
    TF2_WEIGHTS_NAME,
    AutoConfig,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainingArguments,
)

from Modeling import (
    CustomTFTrainer,
    CustomTFTrainingArguments,
    get_eval_metric,
    wrap_relevance_head,
    TFElectraForRelevanceClassification,
)
from args import (
    ModelArguments,
    DataTrainingArguments,
)


logger.set_verbosity(logger.INFO)


training_args = CustomTFTrainingArguments(
    output_dir = OUTPUT_DIR,
    do_train = False,
    do_eval = False,
    do_predict = True,
    do_early_stopping = False,
    evaluate_during_training = False,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 4,
    learning_rate= 3e-6,
    weight_decay = 0.01,
    adam_epsilon = 1e-8,
    num_train_epochs = -1,
    max_steps = 100000,
    warmup_steps = 10000,
    logging_steps = 1000,
    save_steps = 5000,
    tpu_name = f"grpc://{os.environ['COLAB_TPU_ADDR']}",
    ckpt_dir = f'{CHECKPOINT_DIR}/{STRATEGY}', # path to the ckpt 
    eval_all_checkpoints = False,
    logging_dir = LOG_DIR,
)


data_args = DataTrainingArguments(
    collection = COLLECTION,
    how = 'words',
    train_data_dir = TRAIN_DATA_DIR if TRAIN_DATA_DIR else None,
    eval_data_dir = EVAL_DATA_DIR if EVAL_DATA_DIR else None,
    train_set_name = TRAIN_SET_NAME if TRAIN_SET_NAME else None,
    eval_set_name = EVAL_SET_NAME if EVAL_SET_NAME else None,
    test_set_name = TEST_SET_NAME if TEST_SET_NAME else None,
    out_suffix = OUT_SUFFIX,
    eval_qrels_file = EVAL_QRELS_FILE_NAME if EVAL_QRELS_FILE_NAME else None, 
    test_qrels_file = TEST_QRELS_FILE_NAME if TEST_QRELS_FILE_NAME else None,
    max_seq_length = 256,
)
model_args = ModelArguments(
    model_name_or_path = MODEL_NAME_OR_PATH,
)


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    # parser = HfArgumentParser((ModelArguments, DataTrainingArguments, CustomTFTrainingArguments))
    # model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    if (
        tf.io.gfile.exists(training_args.output_dir)
        and tf.io.gfile.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    logger.info(
        "n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.n_gpu,
        bool(training_args.n_gpu > 1),
        training_args.fp16,
    )
    logger.info("Training/evaluation/prediction parameters %s", training_args)

    if training_args.do_train or training_args.do_eval:
       
        num_labels = 2
        output_mode = 'classification'

        # Load pretrained model and tokenizer
        #
        # Distributed training:
        # The .from_pretrained methods guarantee that only one local process can concurrently
        # download model & vocab.

        config = AutoConfig.from_pretrained(
            model_args.config_name if model_args.config_name else model_args.model_name_or_path,
            num_labels=num_labels,
            cache_dir=model_args.cache_dir,
            output_hidden_states=False,
        )

        with training_args.strategy.scope():
            # We use one single layer classifier on top of ELECTRA as in the BERT model we follow: 
            # https://github.com/capreolus-ir/capreolus/blob/master/capreolus/reranker/TFBERTMaxP.py
            # and define a new model TFElectraForRelevanceClassification 
            # if MODEL_TYPE.lower()=='electra':
            #   model = TFElectraForRelevanceClassification.from_pretrained(
            #       model_args.model_name_or_path,
            #       from_pt=True if glob.glob(f"{model_args.model_name_or_path}/*.bin") else False,
            #       config=config,
            #       cache_dir=model_args.cache_dir,
            #   )
            # else:
            #   model = TFAutoModelForSequenceClassification.from_pretrained(
            #       model_args.model_name_or_path,
            #       from_pt=True if glob.glob(f"{model_args.model_name_or_path}/*.bin") else False,
            #       config=config,
            #       cache_dir=model_args.cache_dir,
            #   )

            model = TFAutoModelForSequenceClassification.from_pretrained(
                  model_args.model_name_or_path,
                  from_pt=True if glob.glob(f"{model_args.model_name_or_path}/*.bin") else False,
                  config=config,
                  cache_dir=model_args.cache_dir,
              )
            model = wrap_relevance_head(model)
            


        # Get datasets
        if training_args.do_train:
            filename = os.path.join(data_args.train_data_dir, f'dataset_train_{data_args.train_set_name}.tf')
            if tf.io.gfile.exists(filename):
                train_dataset, num_train_examples = data_args.doc_processor.get_train_dataset(filename, 
                                                                            training_args.train_batch_size, training_args.seed)
            else:
                raise IOError('File does not exist: ', filename)
        else:
            train_dataset, num_train_examples = None, 0

        if training_args.do_eval or training_args.do_early_stopping:
            filename = os.path.join(data_args.eval_data_dir, f'dataset_dev_{data_args.eval_set_name}.tf')
            ids_file = os.path.join(data_args.eval_data_dir, f'query_pass_ids_dev_{data_args.eval_set_name}.tsv')
            if tf.io.gfile.exists(filename):
                eval_dataset, num_eval_examples = data_args.doc_processor.get_eval_dataset(filename, 
                                                                        training_args.eval_batch_size)
            else:
                raise IOError('File does not exist: ', filename)
            if tf.io.gfile.exists(ids_file):
                query_doc_ids = pd.read_csv(ids_file, 
                                            header=None, index_col=None, delimiter='\t', 
                                            names=['id','qid','did','pass'], 
                                            dtype={'id':str, 'qid':str,'did':str})
            else:
                raise IOError('File does not exist: ', ids_file)

            if data_args.eval_qrels_file is not None:
                qrels_file = os.path.join(data_args.eval_data_dir, f'{data_args.eval_qrels_file}.tsv')
                if tf.io.gfile.exists(qrels_file):
                    eval_qrels = pd.read_csv(qrels_file,
                                header=None, index_col=None, delimiter='\t', names=['qid','did','label'], 
                                dtype={'qid':str,'did':str, 'label':int})
                else:
                    raise IOError('File does not exist: ', qrels_file)
            else:
                eval_qrels = None
        else:
            eval_dataset, num_eval_examples, query_doc_ids, eval_qrels = None, 0, None, None

        eval_metric = get_eval_metric(data_args.collection)


        # Initialize our Trainer
        trainer = CustomTFTrainer(
            model = model,
            args = training_args,
            train_dataset = train_dataset,
            num_train_examples = num_train_examples,
            eval_dataset = eval_dataset,
            num_eval_examples = num_eval_examples,
            out_suffix = f'{data_args.eval_set_name}_{data_args.out_suffix}', # eval output file name
            eval_metric = eval_metric,
            compute_metrics = eval_metric.compute_on_df,
            query_doc_ids = query_doc_ids,
            eval_qrels = eval_qrels,
        )

        # Training
        if training_args.do_train:
            trainer.train()

            if not training_args.do_early_stopping:
                trainer.save_model()

        # Evaluation        
        if training_args.do_eval:
            logger.info("*** Evaluate ***")

            result = trainer.evaluate()
            output_eval_file = os.path.join(training_args.output_dir, f"eval_results_{data_args.eval_set_name}.txt")

            with open(output_eval_file, "w") as writer:
                logger.info("***** Eval results *****")

                for key, value in result.items():
                    logger.info("  %s = %s", key, value)
                    writer.write("%s = %s\n" % (key, value))

                results.update(result)
            # trainer.predict(eval_dataset, num_eval_examples, query_doc_ids, eval_qrels, 
            #                 f'{data_args.eval_set_name}_{data_args.out_suffix}-final')

    # Test
    if training_args.do_predict:

        checkpoints = list(
                os.path.dirname(c)
                for c in sorted(
                    glob.glob(f'{training_args.ckpt_dir}' + "/**/" + TF2_WEIGHTS_NAME, recursive=True),
                    key=lambda f: int("".join(filter(str.isdigit, f)) or -1),
                )
            )

        if len(checkpoints) == 0:
            raise IOError('No checkpoint found at this location: ', training_args.ckpt_dir)

        elif not training_args.eval_all_checkpoints:
            if training_args.ckpt_dir in checkpoints:
                checkpoints = [training_args.ckpt_dir]
            else:
                checkpoints = [checkpoints[-1]]    
                    
        logger.info("Evaluate the following checkpoints: %s", checkpoints)

        filename = os.path.join(data_args.eval_data_dir, f'dataset_test_{data_args.test_set_name}.tf')
        if tf.io.gfile.exists(filename):
            test_dataset, num_test_examples = data_args.doc_processor.get_eval_dataset(filename, 
                                                    training_args.eval_batch_size)
        else: 
            raise IOError('File does not exist: ', filename)

        ids_file = os.path.join(data_args.eval_data_dir, f'query_pass_ids_test_{data_args.test_set_name}.tsv')
        if tf.io.gfile.exists(ids_file):
            query_doc_ids = pd.read_csv(ids_file,
                                header=None, index_col=None, delimiter='\t', 
                                names=['id','qid','did','pass'],
                                dtype={'id':str, 'qid':str,'did':str})
        else: 
            raise IOError('File does not exist: ', ids_file)

        if data_args.test_qrels_file:
            qrels_file = os.path.join(data_args.eval_data_dir, f'{data_args.test_qrels_file}.tsv')
            if tf.io.gfile.exists(qrels_file):
                test_qrels = pd.read_csv(qrels_file, 
                            header=None, index_col=None, delimiter='\t', 
                            names=['qid','did','label'], 
                            dtype={'qid':str,'did':str, 'label':int})
                
            else :
                raise IOError('File does not exist: ', qrels_file)
        else:
            test_qrels = None

        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if re.match(".*checkpoint*-[0-9]", checkpoint) else "final"
            logger.info("Evaluate the following checkpoint-step: %s - %s", checkpoint, global_step)
            print("Evaluate the following checkpoint-step:", checkpoint, global_step)

            with training_args.strategy.scope():
                # if MODEL_TYPE.lower()=='electra':
                #   trained_model = TFElectraForRelevanceClassification.from_pretrained(checkpoint)
                # else:
                #   trained_model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
                trained_model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
                trained_model = wrap_relevance_head(trained_model)

            trainer = CustomTFTrainer(
                          model = trained_model,
                          args = training_args,
                      )

            trainer.predict(test_dataset, num_test_examples, query_doc_ids, test_qrels, 
                            f'{data_args.test_set_name}_{data_args.out_suffix}-{global_step}')

if __name__ == "__main__":
    main()

/usr/local/lib/python3.7/dist-packages/transformers/training_args_tf.py:216: FutureWarning: The n_gpu argument is deprecated and will be removed in a future version, use n_replicas instead.
  FutureWarning,


INFO:tensorflow:Initializing the TPU system: grpc://10.24.90.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.24.90.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
INFO:absl:n_gpu: 8, distributed training: True, 16-bits training: False
INFO:absl:Training/evaluation/prediction parameters CustomTFTrainingArguments(output_dir='gs://lila_data/output_dir/zero-shot/robust04/electra', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=True, evaluate_during_training=False, evaluation_strategy='no', prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=3e-06, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=-1, max_steps=100000, warmup_steps=10000, logging_dir='gs://lila_data/log', logging_first_step=False, logging_steps=1000, save_steps=5000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', 

Evaluate the following checkpoint-step: /content/drive/MyDrive/Colab Notebooks/electra_ckpts/sim_pair_wrap_2 final


All model checkpoint weights were used when initializing TFElectraForRelevanceClassification.

All the weights of TFElectraForRelevanceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/electra_ckpts/sim_pair_wrap_2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForRelevanceClassification for predictions without further training.


<class 'Modeling.reRankers.TFElectraForRelevanceClassification'>


INFO:absl:***** Running Prediction *****
INFO:absl:  Num examples = 2213519
INFO:absl:  Batch size = 32
Iteration: 100%|██████████| 69173/69173 [2:33:47<00:00,  7.50it/s]


# Multi_phase

## Parameters

In [ ]:
## Tain params
MODEL_NAME_OR_PATH = "/path/to/msmarco-ft/ckpt" #@param {type:"string"}
#MODEL_TYPE='bert' #@param {type:"string"}

RUNID="test-e1-lr1e5-b32_32"#@param {type:"string"}
OUTPUT_DIR = "/path/output/dir/for/predictions" #@param {type:"string"}
assert OUTPUT_DIR, 'Must specify an existing GCS bucket name'
tf.io.gfile.makedirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Now we need to specify the input data dir. Should contain the .tfrecord files 
# and the supporting query-docids mapping files.

COLLECTION='gov2' #@param {type:"string"}

STRATEGY= "sim_pair" #@param {type:"string"}

FOLD_DATA_DIR = "gs://BUCKET_NAME/gov2/gov2_description_doc_body_p150_o75_256/num-pass-perdoc-30/train_sample_0.1_rawdocs" #@param {type:"string"}

EVAL_QRELS_FILE='gs://path/to/qrels.txt/file' #@param {type:"string"}

CKPT_DIR = "/path/to/save/transformers/ckpts"#@param {type:"string"}

OUT_SUFFIX = "msmarco-indomain"#@param {type:"string"}

TREC_METRICS= "P_20 map ndcg_cut_20"#@param {type:"string"}

LOG_DIR = "gs://BUCKET_NAME/log_dir"#@param {type:"string"}

***** Model output directory: gs://lila_data/output_dir/in_domain/gov2/electra/gov2_title-description_doc_body_p150_o75_256/num-pass-perdoc-30/train_sample_0.1_rawdocs *****


## Run the model

In [ ]:
# coding=utf-8
from absl import logging as logger
import os, glob, re
import pandas as pd
import tensorflow as tf
from transformers import (
    TF2_WEIGHTS_NAME,
    AutoConfig,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainingArguments,
)

from Modeling import (
    CustomTFTrainer,
    CustomTFTrainingArguments,
    get_eval_metric,
    TFElectraForRelevanceClassification,
    wrap_relevance_head,
)
from args import (
    ModelArguments,
    DataTrainingArguments,
)


logger.set_verbosity(logger.INFO)


def get_trec_metrics():
  return set(TREC_METRICS.split())

def main(training_args, data_args, model_args):

    logger.info(
        "n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.n_gpu,
        bool(training_args.n_gpu > 1),
        training_args.fp16,
    )
    logger.info("Training/evaluation/prediction parameters %s", training_args)

    if training_args.do_train or training_args.do_eval:
       
        num_labels = 2
        output_mode = 'classification'

        # Load pretrained model and tokenizer
        #
        # Distributed training:
        # The .from_pretrained methods guarantee that only one local process can concurrently
        # download model & vocab.

        config = AutoConfig.from_pretrained(
            model_args.config_name if model_args.config_name else model_args.model_name_or_path,
            num_labels=num_labels,
            cache_dir=model_args.cache_dir,
            output_hidden_states=False,
        )

        with training_args.strategy.scope():
            # if MODEL_TYPE.lower()=='electra':
            #   model = TFElectraForRelevanceClassification.from_pretrained(
            #       model_args.model_name_or_path,
            #       from_pt=True if glob.glob(f"{model_args.model_name_or_path}/*.bin") else False,
            #       config=config,
            #       cache_dir=model_args.cache_dir,
            #   )
            # else:
            #   model = TFAutoModelForSequenceClassification.from_pretrained(
            #       model_args.model_name_or_path,
            #       from_pt=True if glob.glob(f"{model_args.model_name_or_path}/*.bin") else False,
            #       config=config,
            #       cache_dir=model_args.cache_dir,
            #   )
            model = TFAutoModelForSequenceClassification.from_pretrained(
                  model_args.model_name_or_path,
                  from_pt=True if glob.glob(f"{model_args.model_name_or_path}/*.bin") else False,
                  config=config,
                  cache_dir=model_args.cache_dir,
              )
            model = wrap_relevance_head(model)

        # Get datasets
        if training_args.do_train:
            filename = os.path.join(data_args.train_data_dir, f'dataset_train_{data_args.train_set_name}.tf')

            print('============== Train= ', filename)
            if tf.io.gfile.exists(filename):
                train_dataset, num_train_examples = data_args.doc_processor.get_train_dataset(filename, 
                                                                            training_args.train_batch_size, training_args.seed)
            else:
                raise IOError('File does not exist: ', filename)
        else:
            train_dataset, num_train_examples = None, 0

        if training_args.do_eval or training_args.do_early_stopping:
            filename = os.path.join(data_args.eval_data_dir, f'dataset_test_{data_args.eval_set_name}.tf')
            print('============== Eval= ', filename)
            ids_file = os.path.join(data_args.eval_data_dir, f'query_pass_ids_test_{data_args.eval_set_name}.tsv')
            if tf.io.gfile.exists(filename):
                eval_dataset, num_eval_examples = data_args.doc_processor.get_eval_dataset(filename, 
                                                                        training_args.eval_batch_size)
            else:
                raise IOError('File does not exist: ', filename)
            if tf.io.gfile.exists(ids_file):
                query_doc_ids = pd.read_csv(ids_file, 
                                            header=None, index_col=None, delimiter='\t', 
                                            names=['id','qid','did','pass'], 
                                            dtype={'id':str, 'qid':str,'did':str})
            else:
                raise IOError('File does not exist: ', ids_file)

            if data_args.eval_qrels_file is not None:
                #qrels_file = os.path.join(data_args.eval_data_dir, data_args.eval_qrels_file)
                qrels_file = os.path.join(data_args.eval_qrels_file)
                if tf.io.gfile.exists(qrels_file):
                    eval_qrels = pd.read_csv(qrels_file,
                                header=None, index_col=None, delimiter=' ', names=['qid','_','did','label'], 
                                dtype={'qid':str,'did':str, 'label':int})
                else:
                    raise IOError('File does not exist: ', qrels_file)
            else:
                eval_qrels = None
        else:
            eval_dataset, num_eval_examples, query_doc_ids, eval_qrels = None, 0, None, None

        trec_metrics = get_trec_metrics()

        # Initialize our Trainer
        trainer = CustomTFTrainer(
            model = model,
            args = training_args,
            train_dataset = train_dataset,
            num_train_examples = num_train_examples,
            eval_dataset = eval_dataset,
            num_eval_examples = num_eval_examples,
            out_suffix = f'{data_args.eval_set_name}_{data_args.out_suffix}', # eval output file name
            trec_metrics = trec_metrics,
            query_doc_ids = query_doc_ids,
            eval_qrels = eval_qrels,
        )

        # Training
        if training_args.do_train:
            r = trainer.evaluate(step=0)

            print('step 0')
            print(r)
            trainer.train()

            if not training_args.do_early_stopping:
                trainer.save_model()

        # Evaluation        
        if training_args.do_eval:
            results = trainer.evaluate(step='final')
            print('final')
            print(results)
            
    # Test
    if training_args.do_predict:

        checkpoints = list(
                os.path.dirname(c)
                for c in sorted(
                    glob.glob(f'{training_args.ckpt_dir}' + "/**/" + TF2_WEIGHTS_NAME, recursive=True),
                    key=lambda f: int("".join(filter(str.isdigit, f)) or -1),
                )
            )

        if len(checkpoints) == 0:
            raise IOError('No checkpoint found at this location: ', training_args.ckpt_dir)

        elif not training_args.eval_all_checkpoints:
            if training_args.ckpt_dir in checkpoints:
                checkpoints = [training_args.ckpt_dir]
            else:
                checkpoints = [checkpoints[-1]]    
                    
        logger.info("Evaluate the following checkpoints: %s", checkpoints)

        filename = os.path.join(data_args.eval_data_dir, f'dataset_test_{data_args.test_set_name}.tf')
        if tf.io.gfile.exists(filename):
            test_dataset, num_test_examples = data_args.doc_processor.get_eval_dataset(filename, 
                                                    training_args.eval_batch_size)
        else: 
            raise IOError('File does not exist: ', filename)

        ids_file = os.path.join(data_args.eval_data_dir, f'query_pass_ids_test_{data_args.test_set_name}.tsv')
        if tf.io.gfile.exists(ids_file):
            query_doc_ids = pd.read_csv(ids_file,
                                header=None, index_col=None, delimiter='\t', 
                                names=['id','qid','did','pass'],
                                dtype={'id':str, 'qid':str,'did':str})
        else: 
            raise IOError('File does not exist: ', ids_file)

        if data_args.test_qrels_file:
            qrels_file = os.path.join(data_args.eval_data_dir, data_args.test_qrels_file)
            if tf.io.gfile.exists(qrels_file):
                test_qrels = pd.read_csv(qrels_file, 
                            header=None, index_col=None, delimiter=' ', 
                            names=['qid','_','did','label'], 
                            dtype={'qid':str,'did':str, 'label':int})
                
            else :
                raise IOError('File does not exist: ', qrels_file)
        else:
            test_qrels = None

        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if re.match(".*checkpoint*-[0-9]", checkpoint) else "final"
            logger.info("Evaluate the following checkpoint-step: %s - %s", checkpoint, global_step)
            print("Evaluate the following checkpoint-step:", checkpoint, global_step)

            with training_args.strategy.scope():
                # if MODEL_TYPE.lower()=='electra':
                #   trained_model = TFElectraForRelevanceClassification.from_pretrained(checkpoint)
                # else:
                #   trained_model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
                trained_model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
                trained_model = wrap_relevance_head(trained_model)

            trainer = CustomTFTrainer(
                          model = trained_model,
                          args = training_args,
                      )

            trainer.predict(test_dataset, num_test_examples, query_doc_ids, test_qrels, 
                            f'{data_args.test_set_name}_{data_args.out_suffix}-{global_step}')

if __name__ == "__main__":

    for fold in [1,2,3,4,5]:
        print("============== fold= ",fold)
        print("============== Strategy= ", STRATEGY)

        num_eval_docs=100

        fold_path = f'{FOLD_DATA_DIR}/fold-{fold}-train-1000-test-{num_eval_docs}'

        training_args = CustomTFTrainingArguments(
            output_dir = f'{OUTPUT_DIR}/{RUNID}/fold-{fold}-train-1000-test-{num_eval_docs}',
            do_train = True,
            do_eval = True,
            do_predict = False,
            do_early_stopping = False,
            evaluate_during_training = True,
            per_device_train_batch_size = 4,
            per_device_eval_batch_size = 4,
            learning_rate= 1e-5,
            weight_decay = 0.01,
            adam_epsilon = 1e-6,
            num_train_epochs = 1,
            max_steps = 0,
            warmup_steps = 0,
            warmup_prop = 0.1,
            logging_steps = 1000,
            save_steps = 10000000,
            tpu_name = f"grpc://{os.environ['COLAB_TPU_ADDR']}",
            eval_steps = 25000000,
            patience = 20,
            best_score = None,
            delta = 0,
            ckpt_name = f'{STRATEGY}_fold-{fold}', # see CustomTFTrainingArguments for more details about the ckpts
            ckpt_dir=f"{CKPT_DIR}/{RUNID}/fold-{fold}/ckpt_{STRATEGY}_fold-{fold}", # Where the Transformers library ckpts format will be saved 
            eval_all_checkpoints = False,
            logging_dir = LOG_DIR,
            overwrite_ckpt_dir=True, # Overwirtes the ckpts saved in ckpt_dir if already exsits
            overwrite_tf_ckpt_dir=True, # Overwirtes the TF2 ckpts saved in tf_ckpt_dir if already exsits, i.e Do not continue training from last ckpt
        )

        data_args = DataTrainingArguments(
          collection = COLLECTION,
          how = 'words',
          train_data_dir = f'{FOLD_DATA_DIR}/fold-{fold}-train-1000-test-100',
          eval_data_dir = f'{FOLD_DATA_DIR}/fold-{fold}-train-1000-test-100',
          train_set_name = f'{COLLECTION}-fold-{fold}-{STRATEGY}',
          eval_set_name = f'{COLLECTION}-fold-{fold}-{STRATEGY}_doc_level_marking',
          eval_qrels_file = EVAL_QRELS_FILE,
          out_suffix = OUT_SUFFIX,
          max_seq_length = 256,
        )
        
        model_args = ModelArguments(
            model_name_or_path = f'{MODEL_NAME_OR_PATH}/{STRATEGY}', # initialize with the msmarco fine-tuned ckpt 
        )

        main(training_args, data_args, model_args)

============== fold=  1
============== Strategy=  sim_pair
INFO:tensorflow:Initializing the TPU system: grpc://10.8.155.18:8470


/usr/local/lib/python3.7/dist-packages/transformers/training_args_tf.py:216: FutureWarning: The n_gpu argument is deprecated and will be removed in a future version, use n_replicas instead.
  FutureWarning,
INFO:tensorflow:Initializing the TPU system: grpc://10.8.155.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
INFO:absl:n_gpu: 8, distributed training: True, 16-bits training: False
INFO:absl:Training/evaluation/prediction parameters CustomTFTrainingArguments(output_dir='gs://lila_data/output_dir/in_domain/gov2/electra/gov2_title-description_doc_body_p150_o75_256/num-pass-perdoc-30/train_sample_0.1_rawdocs/test-e1-lr1e5-b32_32/fold-5-train-1000-test-100', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, evaluation_strategy='no', prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=1e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-06, max_grad_norm=1.0, num_train_epochs=1, max_steps=0, warmup_steps=0, logging_dir='gs://lila_data/log', logging_first_step=Fals

============== Train=  gs://lila_data/trec/parade_style/gov2/electra/gov2_title-description_doc_body_p150_o75_256/num-pass-perdoc-30/train_sample_0.1_rawdocs/fold-5-train-1000-test-100/dataset_train_gov2-fold-5-sim_pair.tf
============== Eval=  gs://lila_data/trec/parade_style/gov2/electra/gov2_title-description_doc_body_p150_o75_256/num-pass-perdoc-30/train_sample_0.1_rawdocs/fold-5-train-1000-test-100/dataset_test_gov2-fold-5-sim_pair_doc_level_marking.tf


INFO:absl:***** Running training *****
INFO:absl:  Num examples = 327037
INFO:absl:  Num Epochs = 1
INFO:absl:  Total optimization steps = 10219
INFO:absl:Starting Epoch 1 ...
INFO:absl:Epoch 1 Step 1000 Train Loss 0.4047
INFO:absl:Epoch 1 Step 2000 Train Loss 0.5353
INFO:absl:Epoch 1 Step 3000 Train Loss 0.1734
INFO:absl:Epoch 1 Step 4000 Train Loss 0.2169
INFO:absl:Epoch 1 Step 5000 Train Loss 0.0243
INFO:absl:Epoch 1 Step 6000 Train Loss 0.2782
INFO:absl:Epoch 1 Step 7000 Train Loss 0.0461
INFO:absl:Epoch 1 Step 8000 Train Loss 0.3397
INFO:absl:Epoch 1 Step 9000 Train Loss 0.1316
INFO:absl:Epoch 1 Step 10000 Train Loss 0.1264
INFO:absl:***** Running Evaluation *****
INFO:absl:  Num examples = 57954
INFO:absl:  Batch size = 32
Iteration: 100%|██████████| 1812/1812 [03:36<00:00,  8.36it/s]


final
{'loss': 0.18367758}
